In [1]:
# 设置参数
trainSetPercent = 0.5
num_iterations = 100
learning_rate = 0.001
loadLastTrainData = True
sliceRandomSeed = 0

In [2]:
# 准备数据
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

labels= pd.read_csv('labels_final.csv')
print('data set count:', labels.shape[0])


trainSetCount = math.floor(labels.shape[0] * trainSetPercent)
if trainSetCount < 1: trainSetCount = 1
testSetCount = labels.shape[0] - trainSetCount

print('train set count:', trainSetCount)
print('test set count:', testSetCount)

imgDir = './images/resized/'
imgSize = 0

np.random.seed = sliceRandomSeed
shuffledDataSet = np.array(labels)
# np.random.shuffle(shuffledDataSet)
trainSetRange = shuffledDataSet[:trainSetCount, :]
testSetRange = shuffledDataSet[trainSetCount:trainSetCount + testSetCount, :]

trainSetX = []

for row in trainSetRange:
    img = plt.imread(imgDir + str(row[0]) + '.jpg')
    if imgSize == 0: imgSize = img.size
    elif imgSize != img.size:
        raise ValueError("图片尺寸不一致")
    imgArray = np.array(img)
    imgTrans = imgArray.reshape((1, img.size)).T
    trainSetX.append(imgTrans)

print('image size:', imgSize)

data set count: 5996
train set count: 2998
test set count: 2998
image size: 331776


In [3]:
# 构造训练集
trainSetX = np.array(trainSetX)
trainSetX = trainSetX.squeeze().T / 2550
trainSetY = trainSetRange[:,1:].T

In [4]:
# 构造测试集
testSetX = []

for row in testSetRange:
    img = plt.imread(imgDir + str(row[0]) + '.jpg')
    if imgSize != img.size:
        raise ValueError("图片尺寸不一致")
    imgArray = np.array(img)
    imgTrans = imgArray.reshape((1, img.size)).T
    testSetX.append(imgTrans)

testSetX = np.array(testSetX).squeeze().T / 2550
testSetY = testSetRange[:,1:].T

In [5]:
# 训练
import logicRegression
import utils

savedDataDir = './SavedTraining'
saveDataFilePath = 'savedTrainingData.bin'

if loadLastTrainData: loadedData = utils.loadFromeFile(saveDataFilePath)
else: loadedData = None
if loadedData is None:
    loadedData = {
        'w': np.zeros((trainSetX.shape[0],1)),
        'b': 0,
        'testAccuracy': 0.
    }
else:
    print('last train accuracy: %.2f %%' % loadedData['trainAccuracy'])
    print('last test accuracy: %.2f %%' % loadedData['testAccuracy'])

w = loadedData['w']
b = loadedData['b']
print('trainSetX value type:', trainSetX.dtype)
print('trainSetY value type:', trainSetY.dtype)
print('testSetX value type:', testSetX.dtype)
print('testSetY value type:', testSetY.dtype)
print('w value type:', w.dtype)
print('b value type:', b.dtype)

cost_record_cnt = 100
print_cost = True


last train accuracy: 84.39 %
last test accuracy: 85.06 %
trainSetX value type: float64
trainSetY value type: int64
testSetX value type: float64
testSetY value type: int64
w value type: float64
b value type: float64


In [6]:
import logicRegression

result = logicRegression.modelWithInitialWB(
    trainSetX, trainSetY,
    testSetX, testSetY,
    w, b,
    num_iterations, learning_rate,
    cost_record_cnt, print_cost)

Progress: [....................] cost: 0.3582
Progress: [....................] cost: 0.3582
Progress: [....................] cost: 0.3582
Progress: [....................] cost: 0.3582
Progress: [....................] cost: 0.3582
Progress: [#...................] cost: 0.3582
Progress: [#...................] cost: 0.3582
Progress: [#...................] cost: 0.3582
Progress: [#...................] cost: 0.3582
Progress: [#...................] cost: 0.3582
Progress: [##..................] cost: 0.3582
Progress: [##..................] cost: 0.3582
Progress: [##..................] cost: 0.3582
Progress: [##..................] cost: 0.3582
Progress: [##..................] cost: 0.3582
Progress: [###.................] cost: 0.3582
Progress: [###.................] cost: 0.3582
Progress: [###.................] cost: 0.3582
Progress: [###.................] cost: 0.3582
Progress: [###.................] cost: 0.3582
Progress: [####................] cost: 0.3582
Progress: [####................] c

In [17]:
# 保存模型
saveData = {
    'w': result['w'],
    'b': result['b'],
    'costs': result['costs'],
    'trainAccuracy': result['trainAccuracy'],
    'testAccuracy': result['testAccuracy']

}
utils.save2File(saveData, saveDataFilePath)
print('save to', saveDataFilePath, 'done')

save to savedTrainingData.bin done
